# LightGBM Model Development

**LightGBM** is a fast, distributed, high-performance gradient boosting framework based on decision tree algorithms. It is designed for efficiency and scalability, making it suitable for large datasets and high-dimensional data. LightGBM supports parallel and GPU learning, handles categorical features natively, and often achieves state-of-the-art results in machine learning competitions.

### 1. Load the Datasets

In [ ]:
import pandas as pd
from pathlib import Path

# Directory where the processed data is stored
data_path = Path("../processed_data")

# Load the training and validation datasets
X_train, X_val, y_train, y_val = (
    pd.read_csv(data_path / "X_train.csv"),
    pd.read_csv(data_path / "X_val.csv"),
    pd.read_csv(data_path / "y_train.csv"),
    pd.read_csv(data_path / "y_val.csv")
)

# Dislay the shapes of the datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

### 2. Hyperparameter Tuning & Model Training

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

def objective(trial, X, y):
    """
    Objective function for Optuna to minimize.
    This function trains a LightGBM model with a set of hyperparameters
    suggested by Optuna and returns the cross-validated MAPE.
    """
    # Define the hyperparameter search space for LightGBM
    param = {
        'objective': 'regression_l1',
        'metric': 'mape',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'random_state': 42,
        'n_jobs': -1,
        'verbose':-1
    }

    # Use K-Fold cross-validation to get a robust estimate of the model's performance
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mape_scores = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = lgb.LGBMRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                  callbacks=[lgb.early_stopping(10, verbose=False)])
        preds = model.predict(X_val)
        mape_scores.append(mean_absolute_percentage_error(y_val, preds))

    return np.mean(mape_scores)

In [ ]:
import optuna
import joblib

# Dictionary to store the best models
best_models = {}

# Iterate over each target property to tune and train a model
for target in blend_properties:
    print(f"--- Tuning and Training for {target} ---")
    y = train_df[target]

    # Create an Optuna study to find the best hyperparameters
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X, y), n_trials=50) # 50 trials for a good balance of speed and performance

    # Get the best hyperparameters
    best_params = study.best_params
    print(f"Best MAPE for {target}: {study.best_value}")
    print(f"Best hyperparameters for {target}: {best_params}")

    # Train the final model with the best hyperparameters on the entire training set
    final_model = lgb.LGBMRegressor(**best_params, random_state=42, n_jobs=-1)
    final_model.fit(X, y)

    # Save the trained model to a file
    joblib.dump(final_model, f'models/{target}_model.joblib')
    print(f"Saved best model for {target}")

    best_models[target] = final_model

In [ ]:
# --- Prediction ---
predictions = {}
for target in blend_properties:
    print(f"Predicting {target}...")
    model = best_models[target]
    predictions[target] = model.predict(X_test)

In [ ]:
# --- Create Submission File ---
submission_df = pd.DataFrame({'ID': test_df['ID']})
for target in blend_properties:
    submission_df[target] = predictions[target]

submission_df.to_csv('submission_v2.csv', index=False)
print("\nSubmission file 'submission_v2.csv' created successfully!")
